<br>

# Maps

In [ ]:
#!pip3 install open-geodata --upgrade

In [ ]:
import os
import json
import fiona
import folium
import pandas as pd
import branca as bc
import seaborn as sns
import geopandas as gpd
from datetime import date
from folium import plugins

In [ ]:
from open_geodata import geo, lyr

In [ ]:
from paths import *

In [ ]:
# List geodataframes
geo.get_dataset_names()

gdf = geo.load_dataset('divisa_municipal')

<br>

# Layers

In [ ]:
def add_lyr_aprm(path):
    # Input
    gdf = gpd.read_file(os.path.join(path, '01_aprm.gpkg'))
    gdf = gdf.to_crs(epsg=4326)
    
    # Popup
    #gdf['popup'] = gdf.apply(popup_html_est_aut_empresas, axis=1)

    # Layer
    lyr = folium.GeoJson(
        gdf,
        name='APRMs',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': '#DC143C',
            'color': '#DC143C',
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        # popup=folium.GeoJsonPopup(
        #     ['popup'],
        #     parse_html=False,
        #     max_width='400',
        #     show=False,
        #     labels=False,
        #     sticky=True,            
        # ),
        # marker=folium.Marker(
        #     icon=folium.Icon(
        #         color='lightgray',
        #         icon_color='#FFFF00',
        #         #icon='leaf',
        #     ),
        # ),
        tooltip=folium.GeoJsonTooltip(
            fields=['Manancial'],
            aliases=['Manancial'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )
    return lyr


# PopUp
def popup_html_est_aut_empresas(row):
    # Data
    nome = row['Nome']
    posto = row['Posto']
    url_1 = row['url']
    url_2 = row['url_mapa']
    url_3 = row['url_infoposto']
	
    # Infos
    popup = """
    <div>
    <h5>{}</h5>
    <br>Posto: <b>{}</b>
    <br><a href="{}" target="_blank">url - dados</a></b>
    <br><a href="{}" target="_blank">url - mapa</a></b>
    <br><a href="{}" target="_blank">url - infoposto</a></b>
    </div>
    """.format(nome, posto, url_1, url_2, url_3)
    return popup

<br>

# Mapa Plano Diretor

In [ ]:
def get_map(input_shp):
    # Input
    gdf = gpd.read_file(input_shp)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]
    
    # Zoom
    min_zoom = 10
    max_zoom = 18

    # Create Map
    m = folium.Map(
        #location=list_centroid,
        #zoom_start=10,
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        #zoom_delta=0.1,
        min_lat=bounds[0][0]*(101/100),
        min_lon=bounds[0][1]*(101/100),
        max_lat=bounds[1][0]*(99/100),
        max_lon=bounds[1][1]*(99/100),
        tiles=None,
    )
    
    # Add Base Layers
    m.add_child(lyr.base.google_hybrid(min_zoom, max_zoom))
    m.add_child(lyr.base.google_terrain(min_zoom, max_zoom))
    m.add_child(lyr.base.google_streets(min_zoom, max_zoom))
    m.add_child(lyr.base.google_satellite(min_zoom, max_zoom))
    
    # Plano Diretor
    m.add_child(add_lyr_aprm(output_path_geo))
    
    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False
    ).add_to(m)
    return m

In [ ]:
m = get_map(os.path.join(output_path_geo, '01_aprm.gpkg'))

# Results
m.save(os.path.join(output_path_map, 'map_comite_at.html'))
m